In [1004]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold

In [1005]:
df = pd.read_csv('used_cars_UK.csv')

def remove_outliers(df, col_name):
    z_scores = np.abs((df[col_name] - df[col_name].mean()) / df[col_name].std())
    filtered_df = df[z_scores < 3]
    return filtered_df
print(df.shape)
df=remove_outliers(df, 'Price')
df=df.drop("Service history",axis=1)
df.drop(df.columns[0], axis=1, inplace=True)
df['Previous Owners'].fillna(df['Previous Owners'].value_counts().idxmax(), inplace=True)
df.dropna( inplace=True)
print(df.shape)
df.head


(3685, 14)
(3547, 12)


<bound method NDFrame.head of                title  Price  Mileage(miles)  Registration_Year  \
0        SKODA Fabia   6900           70189               2016   
1     Vauxhall Corsa   1495           88585               2008   
2        Hyundai i30    949          137000               2011   
3         MINI Hatch   2395           96731               2010   
4     Vauxhall Corsa   1000           85000               2013   
...              ...    ...             ...                ...   
3680  Renault Megane   1395           76202               2006   
3681         Audi A4   6990          119000               2012   
3682    BMW 3 Series   3995          139000               2013   
3683    Honda Accord   1390          179190               2007   
3684  Vauxhall Corsa   2000           82160               2013   

      Previous Owners Fuel type  Body type Engine Gearbox  Doors  Seats  \
0                 3.0    Diesel  Hatchback   1.4L  Manual    5.0    5.0   
1                 4.0    Pe

In [1006]:

encoded_data = pd.get_dummies(df[["title",'Fuel type', 'Body type', 'Gearbox']])
df=df.drop(["title",'Fuel type', 'Body type', 'Gearbox'],axis=1)
df=pd.concat([df,encoded_data],axis=1)
df['Emission Class'] = df['Emission Class'].str[5:]
df['Emission Class'] = df['Emission Class'].astype(int)
df['Engine'] = df['Engine'].str[:3]
df['Engine'] = df['Engine'].astype(float)
df["Registration_Year"]=df["Registration_Year"]-df["Registration_Year"].min()
print(df.head())
df.dtypes[:10]

   Price  Mileage(miles)  Registration_Year  Previous Owners  Engine  Doors  \
0   6900           70189                 23              3.0     1.4    5.0   
1   1495           88585                 15              4.0     1.2    3.0   
2    949          137000                 18              2.0     1.4    5.0   
3   2395           96731                 17              5.0     1.4    3.0   
4   1000           85000                 20              2.0     1.3    5.0   

   Seats  Emission Class  title_Abarth 500  title_Abarth 595  ...  \
0    5.0               6             False             False  ...   
1    5.0               4             False             False  ...   
2    5.0               5             False             False  ...   
3    4.0               4             False             False  ...   
4    5.0               5             False             False  ...   

   Body type_Convertible  Body type_Coupe  Body type_Estate  \
0                  False            False      

Price                  int64
Mileage(miles)         int64
Registration_Year      int64
Previous Owners      float64
Engine               float64
Doors                float64
Seats                float64
Emission Class         int32
title_Abarth 500        bool
title_Abarth 595        bool
dtype: object

In [1007]:
x = df.drop("Price",axis=1)
y = df["Price"].values.reshape(df.shape[0],1)

scaler = MinMaxScaler()

x = scaler.fit_transform(x)
y =scaler.fit_transform(y)

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state=42)
x_test.shape

(710, 451)

In [1008]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
import random
random.seed(1)

In [1009]:
def softmax(x):
    exp_scores = np.exp(x - np.max(x))  # subtracting the maximum value for numerical stability
    return exp_scores / np.sum(exp_scores, axis=0)

In [1010]:
def new_child(p1, p2, p3, p4, f=0.8):
    tmp= gen_individual()
    for  i in range(len(p1)):
      difference = f * (p1[i] - p2[i])
      tmp[i] = p3[i] - difference
    if random.random() < 0.9:
        return tmp
    else:
        return p4

In [1011]:
def neuron(w,x, activation_function=np.tanh):
    # return np.sum(w[1:]*x)+w[0]
    # return np.maximum(0, np.sum(w[1:]*x)+w[0])
    return np.activation_function(np.sum(w[1:]*x)+w[0])

In [1012]:
def layer(w,x,numON):
    z=[]
    for i in range(numON):
        z.append(neuron(w[i],x))
    return z

In [1013]:
def hypothesis(w,x,numOL):
    tmp=x
    for i in range(numOL):
        tmp = layer(w[i],tmp,len(w[i]))
    
    return np.sum(tmp*w[-1][0][1:])+w[-1][0][0]

In [1014]:
def cost_function(w,x,y,numOL,m):
    return np.sum((1/2*m)*np.square(hypothesis(w,x,numOL)-y))

In [1015]:
# x=x_train
# y=y_train
# w=[np.array([[1,2,3,4],[3,2,4,5],[3,2,4,5]]),np.array([[6,1,2,2],[2,2,3,4]]),np.array([[6,1,2],[2,3,4]]),np.array([[1,2,3]])]

In [1016]:
def gen_individual():
    tmp1=[]
    l=numfeatures
    for i in range(numlayers):
        tmp=[]
        for z in range(numofneu[i]):
            
            tmp.append(np.random.rand(l+1).tolist())
        l=numofneu[i]
        tmp1.append(np.array(tmp))
    tmp1.append(np.array([(np.random.rand(numofneu[-1]+1)).tolist()]))
    return tmp1

In [1017]:
def gen_population(pop_size):
    pop=[]
    for _ in range(pop_size):
        pop.append(gen_individual())
    return pop

In [1018]:

# Function to perform differential evolution
def differential_evolution(population,x,y,l, f=0.8, cr=0.9):
    new_population = []
    for i, target in enumerate(population):
        a, b, c = random.sample(population, 3)
        mutant = new_child(a, b, c, target, f)
        trial = []
        for j in range(len(target)):
            crossover = random.random()
            if crossover <= cr or j == random.randrange(len(target)):
                trial.append(mutant[j])
            else:
                trial.append(target[j])
        if cost_function(trial, x, y, numlayers, l) < cost_function(target, x, y, numlayers, l):
            new_population.append(trial)
        else:
            new_population.append(target)
    best_solution = min(population, key=lambda f: cost_function(f,x,y,numlayers,l))
    costs = [cost_function(individual, x, y, numlayers, l) for individual in population]
    costs_array = np.array(costs)
    average_cost = np.mean(costs_array)

    print(cost_function(best_solution,x,y,numlayers,l),average_cost)
    return new_population,best_solution

# Sample usage



In [1019]:

numlayers=2
numofneu=[16,16]
numfeatures=451
population = gen_population(100)


In [1022]:
for generation in range(10):
    population,best_solution = differential_evolution(population,x_test,y_test,x_test.shape[0])

13476.831178345683 928530.4670074071
13476.831178345683 784853.9234528398
13476.831178345683 533263.9979418083
13476.831178345683 442756.00123159925
13248.261199553961 374948.89718742674
13248.261199553961 322402.28876251495
13248.261199553961 302883.2197779415
13248.261199553961 277061.62495339615
13248.261199553961 215654.9608409332
13248.261199553961 181520.65774666108


In [1021]:
predictions1=[]
for i in x_test:
    predictions1.append(hypothesis(best_solution,x_test,2))
r2 = r2_score(y_test, predictions1)
print("R2 Score:", r2)

R2 Score: -0.01801130602622525
